In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
##string labeling
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)
##le.inverse_transform(y_train)

NameError: name 'y_train' is not defined

In [3]:
mydata = pd.read_csv("최종전체3.csv",sep=",",encoding = "utf-8")
y=mydata['관객수범주'].values
x_data = mydata.drop('1주차누적관객수', axis=1)
x_data = x_data.drop('1주차최대스크린수', axis=1)
x_data = x_data.drop('관객수범주', axis=1)
x= x_data.values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42,shuffle=True)

In [5]:
params = list(range(10,101,10))

In [6]:
model = RandomForestClassifier(random_state = 42)
gb_param_grid = {
    'n_estimators' : params,
    'max_depth' : params,
    'min_samples_split':params,
    'min_samples_leaf': [1,2,3]
}

cv=StratifiedKFold(n_splits=5, random_state=42)
gb_grid = GridSearchCV(model, param_grid = gb_param_grid,
                      n_jobs=-1,verbose=1,cv=cv)
gb_grid.fit(X_train,y_train)

C:\Users\dltmd\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 3000 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 740 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 1440 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 2340 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 3440 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4740 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 6240 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 7940 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 9840 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 11940 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 14240 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 15000 out of 15000 | elapsed:  4.7min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=False),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [10, 20, 30, 40, 50, 60, 70, 80,
                                               90, 100],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             verbose=1)

In [7]:
gb_grid.best_params_

{'max_depth': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 50}

In [8]:
from sklearn.metrics import classification_report
model = gb_grid.best_estimator_
pred = model.predict(X_test)
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("훈련 세트 정확도 : " +str(train_score))
print("테스트 세트 정확도 : " +str(test_score))
print("\n")
print(classification_report(y_test,pred,digits=4))

훈련 세트 정확도 : 0.8866666666666667
테스트 세트 정확도 : 0.8307692307692308


              precision    recall  f1-score   support

         범주1     0.8889    0.8000    0.8421        20
         범주2     0.7857    0.8148    0.8000        27
         범주3     0.8421    0.8889    0.8649        18

    accuracy                         0.8308        65
   macro avg     0.8389    0.8346    0.8357        65
weighted avg     0.8331    0.8308    0.8309        65



In [9]:
feature_importances = model.feature_importances_
ft_importances = pd.Series(feature_importances,index = x_data.columns)
ft_importances = ft_importances.sort_values(ascending=False)
ft_importances

감독파워             0.408716
배우파워             0.184138
제작사파워            0.145455
개봉전리뷰수           0.114879
감정점수평균           0.050337
등급_전체관람가         0.021247
장르_범죄            0.016360
등급_청소년관람불가       0.009010
등급_15세이상관람가      0.008631
장르_애니메이션         0.007492
등급_12세이상관람가      0.006083
장르_사극            0.005903
장르_액션            0.004739
장르_멜로/로맨스        0.004246
장르_스릴러/공포(호러)    0.004192
장르_드라마           0.003466
장르_코미디           0.003456
장르_미스터리          0.001179
장르_판타지           0.000471
장르_전쟁            0.000000
장르_다큐멘터리         0.000000
dtype: float64

In [ ]:
##회귀
model = gb_grid.best_estimator_
pred = model.predict(X_test)
rmse = mean_squared_error(y_test, pred)**0.5
print("RMSE: %.4f" % rmse)

def MAPE(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

MAPE(y_test,pred)

x1 = range(1,len(y_test)+1)
plt.plot(x1,y_test,'r',label='y_test')
plt.plot(x1,pred,'b',label='y_pred')
plt.legend(loc='upper right')
plt.show()

from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = pd.DataFrame(confusion_matrix(y_test, pred))
sns.heatmap(cm, annot=True)

def dftoCsv(movie_df, num):
    try: 
        movie_df.to_csv(('y'+ str(num) +'.csv'), sep=',', na_rep='NaN', encoding='euc-kr',index = False) 
    except: 
        print("Error")
        
movie_dic = {"y_test": y_test,"y_pred":pred} 
movie_df = pd.DataFrame(movie_dic) 

dftoCsv(movie_df,1)